#### 1. Explore tinkoff api

In [1]:
!ls ../airflow/

airflow.cfg  Dockerfile     plugins	 unittests.cfg
dags	     entrypoint.sh  tinkoff.cfg  webserver_config.py


In [2]:
import tinvest
import pandas as pd
from configparser import ConfigParser
from datetime import datetime, timedelta

In [3]:
config_parser = ConfigParser()

In [4]:
config_parser.read('../airflow/tinkoff.cfg')

['../airflow/tinkoff.cfg']

In [5]:
token = config_parser.get('core', 'TOKEN_TINKOFF')

In [8]:
client = tinvest.SyncClient(token, use_sandbox=True)

In [9]:
client.get_portfolio()

PortfolioResponse(payload=Portfolio(positions=[PortfolioPosition(name='Apple', average_position_price=None, average_position_price_no_nkd=None, balance=Decimal('73'), blocked=Decimal('0'), expected_yield=None, figi='BBG000B9XRY4', instrument_type=<InstrumentType.stock: 'Stock'>, isin='US0378331005', lots=73, ticker='AAPL'), PortfolioPosition(name='Доллар США', average_position_price=None, average_position_price_no_nkd=None, balance=Decimal('86068.86'), blocked=Decimal('0'), expected_yield=None, figi='BBG0013HGFT4', instrument_type=<InstrumentType.currency: 'Currency'>, isin=None, lots=86, ticker='USD000UTSTOM')]), status='Ok', tracking_id='880010526b71ee48')

In [ ]:
client.post_orders_limit_order(
        figi=figi,
        body=tinvest.schemas.LimitOrderRequest(
            lots=lots,
            operation=tinvest.schemas.OperationType(value=op_type),
            price=price
        )
    )

In [7]:
client.clear_sandbox_account()

Empty(payload={}, status='Ok', tracking_id='f0135ae974bbe225')

In [8]:
client.get_portfolio_currencies()

PortfolioCurrenciesResponse(payload=Currencies(currencies=[CurrencyPosition(balance=Decimal('0'), blocked=None, currency=<Currency.eur: 'EUR'>), CurrencyPosition(balance=Decimal('0'), blocked=None, currency=<Currency.rub: 'RUB'>), CurrencyPosition(balance=Decimal('0'), blocked=None, currency=<Currency.usd: 'USD'>)]), status='Ok', tracking_id='d9e2e2396e77a937')

In [9]:
client.set_sandbox_currencies_balance(
    tinvest.schemas.SandboxSetCurrencyBalanceRequest(
        balance=100000,
        currency=tinvest.schemas.SandboxCurrency('USD')
    )
)

Empty(payload={}, status='Ok', tracking_id='2589a528e6858282')

In [10]:
aapl = client.get_market_search_by_ticker('AAPL')

In [11]:
aapl_figi = aapl.payload.instruments[0].figi

In [12]:
data = client.get_market_candles(
    aapl_figi,
    datetime.now() - timedelta(days=365),
    datetime.now(),
    tinvest.schemas.CandleResolution.day,
)

In [26]:
data.payload.candles

[Candle(c=Decimal('81.8'), figi='BBG000B9XRY4', h=Decimal('81.8375'), interval=<CandleResolution.day: 'day'>, l=Decimal('79.96'), o=Decimal('79.96'), time=datetime.datetime(2020, 2, 12, 7, 0, tzinfo=datetime.timezone.utc), v=30336764),
 Candle(c=Decimal('81.2175'), figi='BBG000B9XRY4', h=Decimal('81.865'), interval=<CandleResolution.day: 'day'>, l=Decimal('80.255'), o=Decimal('81.1375'), time=datetime.datetime(2020, 2, 13, 7, 0, tzinfo=datetime.timezone.utc), v=23174296),
 Candle(c=Decimal('81.2375'), figi='BBG000B9XRY4', h=Decimal('81.6125'), interval=<CandleResolution.day: 'day'>, l=Decimal('80.725'), o=Decimal('81.3475'), time=datetime.datetime(2020, 2, 14, 7, 0, tzinfo=datetime.timezone.utc), v=22025920),
 Candle(c=Decimal('79.75'), figi='BBG000B9XRY4', h=Decimal('80.755'), interval=<CandleResolution.day: 'day'>, l=Decimal('77.6'), o=Decimal('80.5125'), time=datetime.datetime(2020, 2, 18, 7, 0, tzinfo=datetime.timezone.utc), v=38141376),
 Candle(c=Decimal('80.905'), figi='BBG000B9X

In [13]:
df = pd.DataFrame(
    data=(
        (
            candle.time,
            candle.o,
            candle.h,
            candle.l,
            candle.c,
            candle.v,
        ) for candle in data.payload.candles
    ),
    columns=(
        'time',
        'open',
        'high',
        'low',
        'close',
        'volume',
    )
)

In [14]:
df.head()

,time,open,high,low,close,volume
0,2020-02-12 07:00:00+00:00,79.96,81.8375,79.96,81.8,30336764
1,2020-02-13 07:00:00+00:00,81.1375,81.865,80.255,81.2175,23174296
2,2020-02-14 07:00:00+00:00,81.3475,81.6125,80.725,81.2375,22025920
3,2020-02-18 07:00:00+00:00,80.5125,80.755,77.6,79.75,38141376
4,2020-02-19 07:00:00+00:00,80.01,81.1375,79.6725,80.905,28958280


In [15]:
df.tail()

,time,open,high,low,close,volume
247,2021-02-04 07:00:00+00:00,136.02,137.46,134.63,137.39,19081589
248,2021-02-05 07:00:00+00:00,137.32,138.4,135.87,136.76,15678428
249,2021-02-08 07:00:00+00:00,137.18,137.91,135,136.91,16655834
250,2021-02-09 07:00:00+00:00,136.01,137.87,135.8,136.01,14578810
251,2021-02-10 07:00:00+00:00,136.51,137,136.02,136.48,29670


#### 2. Save to DB

In [16]:
import psycopg2
from io import StringIO
import csv

In [17]:
df.head()

,time,open,high,low,close,volume
0,2020-02-12 07:00:00+00:00,79.96,81.8375,79.96,81.8,30336764
1,2020-02-13 07:00:00+00:00,81.1375,81.865,80.255,81.2175,23174296
2,2020-02-14 07:00:00+00:00,81.3475,81.6125,80.725,81.2375,22025920
3,2020-02-18 07:00:00+00:00,80.5125,80.755,77.6,79.75,38141376
4,2020-02-19 07:00:00+00:00,80.01,81.1375,79.6725,80.905,28958280


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   time    252 non-null    datetime64[ns, UTC]
 1   open    252 non-null    object             
 2   high    252 non-null    object             
 3   low     252 non-null    object             
 4   close   252 non-null    object             
 5   volume  252 non-null    int64              
dtypes: datetime64[ns, UTC](1), int64(1), object(4)
memory usage: 11.9+ KB


In [19]:
df.loc[:, 'open':'close'] = df.loc[:, 'open':'close'].astype('float16')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   time    252 non-null    datetime64[ns, UTC]
 1   open    252 non-null    float16            
 2   high    252 non-null    float16            
 3   low     252 non-null    float16            
 4   close   252 non-null    float16            
 5   volume  252 non-null    int64              
dtypes: datetime64[ns, UTC](1), float16(4), int64(1)
memory usage: 6.0 KB


In [21]:
login = 'trader'
password = 'income'
host = 'localhost'
port = 5432
dbname = 'stocks'

In [22]:
DB_URL = f'user={login} password={password} host={host} port={port} dbname={dbname}'

In [23]:
DB_URL

'user=trader password=income host=localhost port=5432 dbname=stocks'

In [24]:
def load_df_to_db(df: pd.DataFrame, table_name: str) -> None:
    buffer = StringIO()
    df.to_csv(buffer, index=False, sep='|', na_rep='NUL', quoting=csv.QUOTE_MINIMAL,
             header=False, float_format='%.8f', doublequote=False, escapechar='\\')
    buffer.seek(0)
    copy_query = f"""
        COPY {table_name}({','.join(df.columns)})
        FROM STDIN
        DELIMITER '|'
        NULL 'NUL'
    """
    conn = psycopg2.connect(dsn=DB_URL)
    with conn.cursor() as cursor:
        cursor.copy_expert(copy_query, buffer)
    conn.commit()
    conn.close()

In [25]:
load_df_to_db(df, 'aapl')